In [11]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, cross_val_score

# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [5]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 5) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

# Total 243 songs
def build_training_data(is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return np.array(X), np.array(y)

In [6]:
X, y = build_training_data(True)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X = None
#y = None
#X_train, y_train = np.array(X_train), np.array(y_train)
#X_test, y_test = np.array(X_test), np.array(y_test)

In [7]:
class_weight = {
    0: 1,
    1: 2,
    2: 4,
    3: 8,
    4: 4,
    5: 4,
    6: 6
}

In [9]:
def train_and_test(beat_feature_model):
    print ('Train Error')
    print (beat_feature_model.evaluate(X_train, y_train, batch_size=64))
    print ('Test Error')
    print (beat_feature_model.evaluate(X_test, y_test, batch_size=64))
    print ('\n')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
y_transformed = np.array(list(map(lambda one_hot: np.argmax(one_hot), y)))
gb_clf = GradientBoostingClassifier(n_estimators=5, max_depth=1, verbose=True)
split = 0.75 * len(X)
gb_clf.fit(X[:10000], y_transformed[:10000])

      Iter       Train Loss   Remaining Time 
         1        9789.0957            2.16s


In [27]:
gb_clf.score(X[split:], y_transformed[split:])

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


0.83963224385798019

In [10]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X, y, nb_epoch=50, batch_size=64, verbose=1)
model.save('models/song_class_model.h5')
model = None
gc.collect()

Epoch 1/50
601265/601265 [==============================] - 175s - loss: 0.8015 - acc: 0.8473   
Epoch 2/50
601265/601265 [==============================] - 183s - loss: 0.5107 - acc: 0.8711   
Epoch 3/50
601265/601265 [==============================] - 165s - loss: 0.4587 - acc: 0.8730   
Epoch 4/50
601265/601265 [==============================] - 160s - loss: 0.4362 - acc: 0.8745   
Epoch 5/50
601265/601265 [==============================] - 161s - loss: 0.4237 - acc: 0.8755   
Epoch 6/50
601265/601265 [==============================] - 169s - loss: 0.4150 - acc: 0.8763   
Epoch 7/50
601265/601265 [==============================] - 158s - loss: 0.4087 - acc: 0.8767   
Epoch 8/50
601265/601265 [==============================] - 142s - loss: 0.4036 - acc: 0.8777   
Epoch 9/50
601265/601265 [==============================] - 145s - loss: 0.3999 - acc: 0.8781   
Epoch 10/50
601265/601265 [==============================] - 144s - loss: 0.3966 - acc: 0.8785   
Epoch 11/50
601265/601265 [==

NameError: name 'X_train' is not defined

In [ ]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X, y, nb_epoch=50, batch_size=64, verbose=1, class_weight=class_weight)
model.save('models/song_class_model_weighted.h5')
model = None
gc.collect()